In [1]:
from typing import List, Dict, Any

from pathlib import Path
import os, sys
import time
import datetime as dt
import itertools as it

import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm
import ray
from ray import train, tune
import ipywidgets as widgets

from recsys4daos.model_selection import cvtt_open
from recsys4daos.datasets import to_microsoft, filter_window_size
from recsys4daos.models import LightGCNCustom
from recsys4daos.utils import Timer
import recsys4daos.utils.notebooks as nbu

import recommenders
if recommenders.__version__ == '1.2.0':
    print("Ignoring warnings")
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)

import paths

nbu.print_versions('ray', 'tensorflow')

%load_ext autoreload
%autoreload 2

/tmp/ipykernel_3805857/1371776918.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Ignoring warnings
3.11.6 (main, Jun 24 2024, 07:45:26) [GCC 11.4.0]
recommenders: 1.2.0
ray: 2.30.0
tensorflow: 2.16.1


In [2]:
# Others config
SEED: int = 57
RAY_RESULTS_PATH: Path = '~/ray_results3.11'

# Dataset splits config
ORG_NAME = 'Decentraland'
SPLITS_FREQ = 'W-THU'  # Split weekly
LAST_FOLDS = 10  # Use just last 10 splits
SPLITS_NORMALIZE = True
LAST_FOLD_DATE_STR: str = '2023-07-13'

# Training config
MAX_EPOCHS: int = 200
EPOCHS_PER_ITER: int = 5
SAMPLES_PER_SPLIT: int = 100
MAX_TIME_TOTAL_TRAIN: int = 300
OPTIM_METRIC: str = 'map@10'

# Search space config
MAX_EMBEDDING_DIM = 1024
MIN_BATCH_SIZE = 6
MAX_BATCH_SIZE = 10 # 2**10
MIN_LR = 1e-4
# WINDOW_SIZES = ['7d', '14d', '21d', '30d', '60d', '90d', '10YE']
WINDOW_SIZES = ['21d', '30d', '60d', '90d', '10YE']
GPUS = 16

# Eval config
K_RECOMMENDATIONS: List[int] = [1,3,5,10,15,100]
METRICS: List[str] = ["recall", "ndcg", "precision", "map"]

In [3]:
# Parameters
EXECUTION_ID = "2024-09-04T10:00"
MAX_BATCH_SIZE = 9
ORG_NAME = "PancakeSwap"
SPLITS_FREQ = "3d"
LAST_FOLDS = 10
SPLITS_NORMALIZE = True
LAST_FOLD_DATE_STR = "2023-06-27"


In [4]:
RAY_RESULTS_PATH = Path(RAY_RESULTS_PATH).expanduser()

## Obtain dataset

In [5]:
!pwd

/home/daviddavo/recsys4daos/notebooks


In [6]:
dfp = paths.load_proposals(ORG_NAME)
dfv = paths.load_votes(ORG_NAME)

print(dfp.info())
print(dfv.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3119 entries, 0 to 3118
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 3119 non-null   object        
 1   author             3119 non-null   object        
 2   date               3119 non-null   datetime64[us]
 3   start              3119 non-null   datetime64[us]
 4   end                3119 non-null   datetime64[us]
 5   platform_proposal  3119 non-null   object        
dtypes: datetime64[us](3), object(3)
memory usage: 146.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532830 entries, 0 to 532829
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        532830 non-null  object        
 1   proposal  532830 non-null  object        
 2   voter     532830 non-null  object        
 3   date      532830 non-null  datetime64[

## Split data

In [7]:
df = to_microsoft(dfv)
df

,userID,itemID,timestamp,rating
0,0x8b017905dc96b38f817473dc885f84d4c76bc113,c3d4fe9b-5193-5b39-a85d-60cdf62f8ebe,2021-06-28 12:14:46,1
1,0x9e882c739493886ab2865803432d3140b6473bfc,c3d4fe9b-5193-5b39-a85d-60cdf62f8ebe,2021-06-28 12:41:42,1
2,0x1e5c8c04e56f88c09ad775e43a164de264d6dad7,c3d4fe9b-5193-5b39-a85d-60cdf62f8ebe,2021-06-28 12:42:56,1
3,0x977e0c1005dff8749f8cac22f4df0bd5f013d1a7,c3d4fe9b-5193-5b39-a85d-60cdf62f8ebe,2021-06-28 12:54:31,1
4,0x1ce49ed469b2c0edf50b574cea9a196bbe23655b,c3d4fe9b-5193-5b39-a85d-60cdf62f8ebe,2021-06-28 16:02:34,1
...,...,...,...,...
532825,0xd554664fb7e5c4ad17a5f32343843f30eb4c093d,718fe10e-25cf-5c87-b966-99e1ca411fb1,2023-07-20 03:06:58,1
532826,0x71c9137eb29ce52e7bdab9ee5cac25208f910665,718fe10e-25cf-5c87-b966-99e1ca411fb1,2023-07-20 03:16:03,1
532827,0x843146d71d57af0e731f85db1f094643776c8e91,718fe10e-25cf-5c87-b966-99e1ca411fb1,2023-07-20 03:33:17,1
532828,0x8dcad3b779415f519c21bb42c9a6743e8cef1c5d,718fe10e-25cf-5c87-b966-99e1ca411fb1,2023-07-20 03:47:32,1


In [8]:
# USE INTEGER INDEX
# folds_dict = list(cvtt_open(df, SPLITS_FREQ, dfp, remove_not_in_train_col='userID'))
# use_folds_idx = range(len(folds_dict))[-LAST_FOLDS:]

# USE TIMESTAMP INDEX
# Note: NO need to used OrderedDict, dict is ordered since Python 3.6
folds_dict = { f.end.isoformat():f for f in cvtt_open(df, SPLITS_FREQ, dfp, remove_not_in_train_col='userID', last_fold=LAST_FOLD_DATE_STR) }
use_folds_idx = list(folds_dict.keys())[-LAST_FOLDS:]

print(len(folds_dict), "folds")
print("Using", len(use_folds_idx), "folds, from", use_folds_idx[0], "to", use_folds_idx[-1])

244 folds
Using 10 folds, from 2023-05-31T00:00:00 to 2023-06-27T00:00:00


## Making some checks

### Checking that all the folds have open proposals

In [9]:
_nok_folds = []
for idx in use_folds_idx:
    if len(folds_dict[idx].open_proposals) == 0:
        _nok_folds.append(idx)

if _nok_folds:
    raise ValueError(f"Folds {', '.join(_nok_folds)} don't have any open proposals")

### Checking correct hparams

In [10]:
now = dt.datetime.now()
max_bs = 2**MAX_BATCH_SIZE

all_ok = True
for idx in use_folds_idx:
    valid_ws = WINDOW_SIZES[0]
    for ws in sorted(WINDOW_SIZES, key=lambda ws: now + pd.tseries.frequencies.to_offset(ws)):
        nusers = filter_window_size(folds_dict[idx].train, folds_dict[idx].end, ws)['userID'].nunique()
        if nusers > max_bs:
            print(f'On fold {idx} WINDOW_SIZE should be at least {ws}: {nusers} > {max_bs}', file=sys.stderr)
            break

print("All folds ok!")

All folds ok!


On fold 2023-05-31T00:00:00 WINDOW_SIZE should be at least 21d: 3008 > 512
On fold 2023-06-03T00:00:00 WINDOW_SIZE should be at least 21d: 3324 > 512
On fold 2023-06-06T00:00:00 WINDOW_SIZE should be at least 21d: 3220 > 512
On fold 2023-06-09T00:00:00 WINDOW_SIZE should be at least 21d: 3017 > 512
On fold 2023-06-12T00:00:00 WINDOW_SIZE should be at least 21d: 3177 > 512
On fold 2023-06-15T00:00:00 WINDOW_SIZE should be at least 21d: 3319 > 512
On fold 2023-06-18T00:00:00 WINDOW_SIZE should be at least 21d: 3611 > 512
On fold 2023-06-21T00:00:00 WINDOW_SIZE should be at least 21d: 3384 > 512
On fold 2023-06-24T00:00:00 WINDOW_SIZE should be at least 21d: 2982 > 512
On fold 2023-06-27T00:00:00 WINDOW_SIZE should be at least 21d: 3082 > 512


## Testing model

In [11]:
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

In [12]:
# hparams = prepare_hparams(
#     model_type='lightgcn',
#     n_layers=3,
#     batch_size=512,
#     embed_size=64,
#     epochs=2,
#     learning_rate=0.001,
#     decay=0.001,
#     metrics=["recall", "ndcg", "precision", "map"],
#     eval_epoch=2,
#     top_k=K_RECOMMENDATIONS[0],
#     save_model=False,
#     MODEL_DIR='./data/model/lightgcn/',
# )
# dataloader = ImplicitCF(train=folds_dict[use_folds_idx[0]].train, test=folds_dict[use_folds_idx[0]].test, seed=SEED)
# print("items:", dataloader.n_items, "user:", dataloader.n_users)
# model = LightGCNCustom(data=dataloader, hparams=hparams)

In [13]:
# model.fit()
# model.run_eval()

In [14]:
# model.recommend_k_items(
#     dataloader.test, 
#     top_k=3, 
#     use_id=True, 
#     remove_seen=True, 
#     recommend_from=folds_dict[use_folds_idx[0]].open_proposals
# )

## Defining trainable

In [15]:
from recsys4daos.evaluation import calculate_all_metrics

In [16]:
class TrainLightGCN(tune.Trainable):
    def setup(
        self,
        config: Dict[str, Any],
        data,
    ):
        self.config = config
        train, test, self.t, self.open_proposals = data
        train_filtered = filter_window_size(train, self.t, config['window_size'])
        self.dataloader = ImplicitCF(train=train_filtered, test=test, seed=SEED)
        # Some experiments will run multiple times, but that's a price to pay for
        # usability
        self.real_batch_size = min(2**config['batch_size'], self.dataloader.n_users_in_train)

        self.hparams = prepare_hparams(
            model_type='lightgcn',
            n_layers=config['conv_layers'],
            batch_size=self.real_batch_size,
            embed_size=config['embedding_dim'],
            epochs=EPOCHS_PER_ITER,
            learning_rate=config['learning_rate'],
            decay=config['l2'],
            metrics=METRICS,
            eval_epoch=-1,
            top_k=K_RECOMMENDATIONS[0],
            save_model=False,
            MODEL_DIR='./data/model/lightgcn/',
        )
        self.model = LightGCNCustom(self.hparams, self.dataloader, seed=SEED)
        self.total_train = 0
        self.total_eval = 0

    @property
    def iteration(self):
        return self.model.epochs_done

    @property
    def training_iteration(self):
        return self.model.epochs_done

    def step(self):
        """
        As a rule of thumb, the execution time of step should be large enough to avoid overheads 
        (i.e. more than a few seconds), but short enough to report progress periodically 
        (i.e. at most a few minutes).
        """
        assert EPOCHS_PER_ITER > 0

        with Timer() as t_train:
            for _ in range(EPOCHS_PER_ITER):
                ret = self.model.fit_epoch()


        with Timer() as t_rec:
            recs = self.model.recommend_k_items(
                self.dataloader.test, # Used only to get user ids
                top_k=max(K_RECOMMENDATIONS),
                use_id=True,
                remove_seen=True,
                recommend_from=self.open_proposals,
            )
        
        eval_dict = {'model_'+k:v for k,v in zip(self.model.metrics, self.model.run_eval())}
        eval_dict |= calculate_all_metrics(self.dataloader.test, recs, K_RECOMMENDATIONS)

        self.total_train += t_train.time
        self.total_eval += eval_dict['time_eval']
        
        return {
            'real_batch_size': self.real_batch_size,
            'iteration': self.iteration,
            'loss': ret[0],
            'mf_loss': ret[1],
            'emb_loss': ret[2],
            **eval_dict,
            'time_train': t_train.time,
            'time_rec': t_rec.time,
            'time_total_train': self.total_train,
            'time_total_test': self.total_eval,
        }

    def save_checkpoint(self, checkpoint_dir):
        checkpoint_path = os.path.join(checkpoint_dir, "model")
        self.model.saver.save(
            sess=self.model.sess,
            save_path=checkpoint_path,
        )
        return checkpoint_dir

    def load_checkpoint(self, checkpoint_path):
        self.model.load(checkpoint_path)

## Big experiment

In [17]:
RAY_RESULTS_PATH

PosixPath('/home/daviddavo/ray_results3.11')

In [18]:
print(os.uname().nodename)

### SET TRAINING RESOURCES
if os.uname().nodename == 'lamarck':
    # assert torch.cuda.is_available()

    NUM_SAMPLES = SAMPLES_PER_SPLIT
    # Every run takes approx half a gig of vram (no optimizations)
    # The RTX 4090 has 24GB so we can run the model about 48 times
    resources_per_trial={
        'cpu': 1,
        'gpu': 1 / GPUS,
    }
else:
    NUM_SAMPLES = 1
    resources_per_trial={
        'cpu': 1,
        # It takes about 1.5 GiB with full training data, but I put a bit more because
        # this notebook also takes a bit of memory
        'memory': 2e9,
    }
print(resources_per_trial)

lamarck
{'cpu': 1, 'gpu': 0.0625}


In [19]:
from ray.tune.search.hyperopt import HyperOptSearch

In [20]:
def getTunerOnFold(f_idx, points_to_evaluate = None):    
    name = paths.lightgcn_ray_tune_fname(ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, OPTIM_METRIC, fold=f_idx)
    experiments = list(RAY_RESULTS_PATH.glob(f'{name}_*'))
    last_experiment = max(experiments, key=lambda x: x.stat().st_ctime) if experiments else None
    f = folds_dict[f_idx]

    dftrain,dftest,t,open_proposals = folds_dict[f_idx]
    param_space = dict(
        batch_size=tune.randint(MIN_BATCH_SIZE, MAX_BATCH_SIZE+1), # 64 - 2**MAX_BATCH_SIZE
        embedding_dim=tune.lograndint(1, MAX_EMBEDDING_DIM, base=2),
        conv_layers=tune.randint(1,5),
        learning_rate=tune.qloguniform(MIN_LR, 1, 1e-4),
        l2=tune.loguniform(1e-7, 1e-2, 1e-7),
        window_size=tune.choice(WINDOW_SIZES),
        # Just so it appears on the output
        fold=f_idx,
    )
    
    ### RESTORE EXPERIMENT OR CREATE A NEW ONE
    if last_experiment and tune.Tuner.can_restore(last_experiment):
        print(f"Restoring last experiment: {last_experiment}")
        tuner = tune.Tuner.restore(
            str(last_experiment),
            trainable=tune.with_resources(
                # tune.with_parameters(TrainLightGCN,  train=dftrain, test=dftest, open_proposals=open_proposals),
                tune.with_parameters(TrainLightGCN, data=f),
                resources_per_trial,
            ),
            restart_errored=True,
            param_space=param_space,
        )
    else:
        print(f"No experiment found for fold {f_idx}, creating new tuner with {NUM_SAMPLES} samples")
        search_alg = None
        
        search_alg = HyperOptSearch(
            points_to_evaluate = points_to_evaluate,
            random_state_seed=SEED,
        )
        # search_alg = tune.search.Repeater(search_alg, N_SPLITS-SKIP_SPLIT)
        
        tuner = tune.Tuner(
            tune.with_resources(
                # tune.with_parameters(TrainLightGCN,  train=dftrain, test=dftest, open_proposals=open_proposals),
                tune.with_parameters(TrainLightGCN, data=folds_dict[f_idx]),
                resources_per_trial,
            ),
            run_config=train.RunConfig(
                stop={'training_iteration': MAX_EPOCHS/EPOCHS_PER_ITER, 'time_total_train': MAX_TIME_TOTAL_TRAIN},
                name=name + f'_{dt.datetime.now().isoformat()}',
                storage_path=RAY_RESULTS_PATH,
                # failure_config=train.FailureConfig(fail_fast='raise'),
                failure_config=train.FailureConfig(max_failures=3),
            ),
            param_space=param_space,
            tune_config=tune.TuneConfig(
                search_alg=search_alg,
                num_samples=NUM_SAMPLES,
                metric=OPTIM_METRIC,
                mode='max',
            )
        )

    return tuner

In [21]:
# We need to display the progress bar in another cell because ray tune "overwrites" the previous output
pbar = tqdm(total=len(use_folds_idx), desc='fold')
out = widgets.Output(layout={'border': '1px solid black'})
with out:
    print("In this cell important output from the next cell will be shown")
out

fold:   0%|          | 0/10 [00:00<?, ?it/s]

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [22]:
import logging
import requests

def findConfig(rg):
    for r in rg:
        if r.config:
            lbrc = last_best_result.config
            if all((r.config[k] == v for k, v in last_best_result.config.items() if k != 'fold')):
                return r
            elif all((r.config[k] == v for k, v in last_best_result.config.items() if k != 'fold' and k != 'window_size')):
                print("Possible coincidence:", r.config, file=sys.stderr)

    return None

tuners = []
results = []
last_best_result = None
pbar.reset()

last_best_fold = None
requests.post("https://ntfy.sh/grasia_notebooks", data=f"Start running microsoft_tuning for {ORG_NAME}")
for i, (prev_f_idx, f_idx) in enumerate(zip(it.chain([None], use_folds_idx), use_folds_idx)):
    try:
        with out:
            best_prev_config = None
            if last_best_result is not None:
                best_prev_config = last_best_result.config.copy()
                assert best_prev_config['fold'] == prev_f_idx
                best_prev_config['fold'] = f_idx
                print(f"Also evaluating best_prev_config ({OPTIM_METRIC}={last_best_result.metrics[OPTIM_METRIC]}): {best_prev_config}")
                best_prev_config = [best_prev_config]
        
        t = getTunerOnFold(f_idx, best_prev_config)
        tuners.append(t)
    
        rg = t.fit()
        
        # FIXME: load results from disk until ray-project/ray#47358 is solved
        # https://github.com/ray-project/ray/issues/47358
        t = getTunerOnFold(f_idx, best_prev_config)
        tuners[-1] = t
        rg = t.fit()
    
        assert rg.num_errors == 0, f"There are {rg.num_errors} errors"
        assert rg.num_terminated >= NUM_SAMPLES, f'Some samples are not terminated ({rg.num_terminated} != {NUM_SAMPLES})'
        assert len(rg.get_dataframe()) >= NUM_SAMPLES
        results.append(rg)
    
        # Assert that the prev config has been tried
        if last_best_result is not None:
            # if not any( 
            #     all((r.config[k] == v for k, v in last_best_result.config.items() if k != 'fold'))
            #     for r in rg if r.config
            # ):
            if not findConfig(rg):
                print("Best config:", last_best_result.config)
                assert False, f"The best config from previous fold has not been tested in fold {f_idx}"    
            else:
                logging.info(f'Fold {f_idx}. Best prev result was {last_best_result.path} and config has been found {findConfig(rg).path}')
        
        last_best_result = rg.get_best_result()
        pbar.update()
    
        print(f"Finished training for fold {f_idx}")
        requests.post("https://ntfy.sh/grasia_notebooks", data=f"Finished running fold {i} for {ORG_NAME}")
    except Exception as e:
        requests.post("https://ntfy.sh/grasia_notebooks", data=f"Error on fold {i} for {ORG_NAME}: {repr(e)}")
        raise

pbar.close()

2024-09-05 08:56:08,267	INFO tune_controller.py:444 -- Restoring the run from the latest experiment state file: experiment_state-2024-09-05_08-56-07.json


2024-09-05 08:56:08,967	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/daviddavo/ray_results3.11/PancakeSwap/LightGCN_3d_normalize_map@10_fold=2023-06-27T00:00:00_2024-08-30T17:27:29.962172' in 0.3634s.


2024-09-05 08:56:08,980	INFO tune.py:1041 -- Total run time: 0.72 seconds (0.00 seconds for the tuning loop).


Finished training for fold 2023-06-27T00:00:00


In [23]:
requests.post("https://ntfy.sh/grasia_notebooks", data=f"Finished running everything for {ORG_NAME}")
print("Finished!")

Finished!
